In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from timple.timedelta import strftimedelta
import numpy as np

import fastf1
import fastf1.plotting as f1plot
from fastf1.core import Laps
import seaborn as sns
from fastf1.ergast import Ergast
ergast = Ergast()

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score, mean_absolute_error
from scipy.stats import spearmanr
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

# black background
f1plot.setup_mpl(mpl_timedelta_support=True, misc_mpl_mods=False,
                          color_scheme='fastf1')

# tire compound colors
compound_colors = {'TEST_UNKNOWN': 'grey', 'SOFT': 'red', 'MEDIUM': 'yellow', 'HARD': 'white', 'INTER': 'green', 'WET': 'blue'}



year = 2024

In [2]:
schedule = fastf1.get_event_schedule(year)
schedule = schedule[schedule['EventFormat'] == 'conventional']
schedule.reset_index(drop=True)

req         WARNING 	DEFAULT CACHE ENABLED! (4.16 GB) /Users/EmilyHuang/Library/Caches/fastf1


,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,2024-03-02,Bahrain Grand Prix,conventional,Practice 1,2024-02-29 14:30:00+03:00,2024-02-29 11:30:00,...,Practice 3,2024-03-01 15:30:00+03:00,2024-03-01 12:30:00,Qualifying,2024-03-01 19:00:00+03:00,2024-03-01 16:00:00,Race,2024-03-02 18:00:00+03:00,2024-03-02 15:00:00,True
1,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2024,2024-03-09,Saudi Arabian Grand Prix,conventional,Practice 1,2024-03-07 16:30:00+03:00,2024-03-07 13:30:00,...,Practice 3,2024-03-08 16:30:00+03:00,2024-03-08 13:30:00,Qualifying,2024-03-08 20:00:00+03:00,2024-03-08 17:00:00,Race,2024-03-09 20:00:00+03:00,2024-03-09 17:00:00,True
2,3,Australia,Melbourne,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2024,2024-03-24,Australian Grand Prix,conventional,Practice 1,2024-03-22 12:30:00+11:00,2024-03-22 01:30:00,...,Practice 3,2024-03-23 12:30:00+11:00,2024-03-23 01:30:00,Qualifying,2024-03-23 16:00:00+11:00,2024-03-23 05:00:00,Race,2024-03-24 15:00:00+11:00,2024-03-24 04:00:00,True
3,4,Japan,Suzuka,FORMULA 1 MSC CRUISES JAPANESE GRAND PRIX 2024,2024-04-07,Japanese Grand Prix,conventional,Practice 1,2024-04-05 11:30:00+09:00,2024-04-05 02:30:00,...,Practice 3,2024-04-06 11:30:00+09:00,2024-04-06 02:30:00,Qualifying,2024-04-06 15:00:00+09:00,2024-04-06 06:00:00,Race,2024-04-07 14:00:00+09:00,2024-04-07 05:00:00,True
4,7,Italy,Imola,FORMULA 1 MSC CRUISES GRAN PREMIO DEL MADE IN ...,2024-05-19,Emilia Romagna Grand Prix,conventional,Practice 1,2024-05-17 13:30:00+02:00,2024-05-17 11:30:00,...,Practice 3,2024-05-18 12:30:00+02:00,2024-05-18 10:30:00,Qualifying,2024-05-18 16:00:00+02:00,2024-05-18 14:00:00,Race,2024-05-19 15:00:00+02:00,2024-05-19 13:00:00,True
5,8,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2024,2024-05-26,Monaco Grand Prix,conventional,Practice 1,2024-05-24 13:30:00+02:00,2024-05-24 11:30:00,...,Practice 3,2024-05-25 12:30:00+02:00,2024-05-25 10:30:00,Qualifying,2024-05-25 16:00:00+02:00,2024-05-25 14:00:00,Race,2024-05-26 15:00:00+02:00,2024-05-26 13:00:00,True
6,9,Canada,Montréal,FORMULA 1 AWS GRAND PRIX DU CANADA 2024,2024-06-09,Canadian Grand Prix,conventional,Practice 1,2024-06-07 13:30:00-04:00,2024-06-07 17:30:00,...,Practice 3,2024-06-08 12:30:00-04:00,2024-06-08 16:30:00,Qualifying,2024-06-08 16:00:00-04:00,2024-06-08 20:00:00,Race,2024-06-09 14:00:00-04:00,2024-06-09 18:00:00,True
7,10,Spain,Barcelona,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2024,2024-06-23,Spanish Grand Prix,conventional,Practice 1,2024-06-21 13:30:00+02:00,2024-06-21 11:30:00,...,Practice 3,2024-06-22 12:30:00+02:00,2024-06-22 10:30:00,Qualifying,2024-06-22 16:00:00+02:00,2024-06-22 14:00:00,Race,2024-06-23 15:00:00+02:00,2024-06-23 13:00:00,True
8,12,United Kingdom,Silverstone,FORMULA 1 QATAR AIRWAYS BRITISH GRAND PRIX 2024,2024-07-07,British Grand Prix,conventional,Practice 1,2024-07-05 12:30:00+01:00,2024-07-05 11:30:00,...,Practice 3,2024-07-06 11:30:00+01:00,2024-07-06 10:30:00,Qualifying,2024-07-06 15:00:00+01:00,2024-07-06 14:00:00,Race,2024-07-07 15:00:00+01:00,2024-07-07 14:00:00,True
9,13,Hungary,Budapest,FORMULA 1 HUNGARIAN GRAND PRIX 2024,2024-07-21,Hungarian Grand Prix,conventional,Practice 1,2024-07-19 13:30:00+02:00,2024-07-19 11:30:00,...,Practice 3,2024-07-20 12:30:00+02:00,2024-07-20 10:30:00,Qualifying,2024-07-20 16:00:00+02:00,2024-07-20 14:00:00,Race,2024-07-21 15:00:00+02:00,2024-07-21 13:00:00,True


In [3]:
import os
def season_history(year):
    if os.path.exists(f'{year}/quali_history.csv'):
        return pd.read_csv(f'{year}/quali_history.csv', index_col=0)
    # Get the event schedule for the season
    schedule = fastf1.get_event_schedule(year)

    # Initialize an empty dictionary to store qualifying results
    qualifying_data = {}

    # Loop through each race in the schedule
    for round_number in schedule['RoundNumber'][1:-1]:
        try:
            # Load qualifying session data for the race
            event = schedule[schedule['RoundNumber'] == round_number]
            grand_prix = event['EventName'].iloc[0]
            session = fastf1.get_session(year, grand_prix, 'Q')
            session.load()

            # Extract qualifying results
            results = session.results[['Abbreviation', 'Position']]

            # Add the round number as a column to the results
            results.rename(columns={'Position': f'Round {round_number}'}, inplace=True)
            results.set_index('Abbreviation', inplace=True)

            # Merge the results into the qualifying_data dictionary
            qualifying_data[round_number] = results[f'Round {round_number}']

        except Exception as e:
            print(f"Could not process Round {round_number}: {e}")

    # Combine all rounds into a single DataFrame
    qualifying_df = pd.concat(qualifying_data, axis=1)

    # Sort the DataFrame by driver abbreviation
    qualifying_df = qualifying_df.sort_index()

    # Save the DataFrame to a CSV file
    qualifying_df.to_csv(f'{year}/quali_history.csv')
    return qualifying_df

qualifying_history = season_history(year)

In [4]:
import fastf1
import pandas as pd
import numpy as np
import os

class Race:
    def __init__(self, event_name, year=None) -> None:
        # If year is not provided, try to infer it from the schedule
        if year is None:
            # Assume 'schedule' is a global DataFrame with columns 'EventName' and 'Year'
            try:
                year = schedule[schedule['EventName'] == event_name]['Year'].values[0]
            except (NameError, IndexError):
                # Fallback to current year if schedule is not available
                import datetime
                year = datetime.datetime.now().year

        self.event_name = event_name
        self.year = year
        self.directory = f'{self.year}/{event_name}'

        # Implement more robust error handling for data retrieval
        try:
            # Check if data files exist, if not, load and save them
            if self.data_files_exist():
                self.load_data_from_files()
            else:
                self.fp1 = self.get_fp(1)
                self.fp2 = self.get_fp(2)
                self.fp3 = self.get_fp(3)
                self.quali = self.get_qualifying()
                self.save_dataframes()

            self.drivers = self.quali['Abbreviation'].unique()
            self.teams = self.quali['TeamName'].unique()
        
        except Exception as e:
            print(f"Error initializing Race data for {event_name}: {e}")
            raise

    def data_files_exist(self):
        return (os.path.exists(f'{self.directory}/fp1.csv') and
                os.path.exists(f'{self.directory}/fp2.csv') and
                os.path.exists(f'{self.directory}/fp3.csv') and
                os.path.exists(f'{self.directory}/quali.csv'))

    def load_data_from_files(self):
        self.fp1 = pd.read_csv(f'{self.directory}/fp1.csv')
        self.fp2 = pd.read_csv(f'{self.directory}/fp2.csv')
        self.fp3 = pd.read_csv(f'{self.directory}/fp3.csv')
        self.quali = pd.read_csv(f'{self.directory}/quali.csv')

    def save_dataframes(self):
        os.makedirs(self.directory, exist_ok=True)
        self.fp1.to_csv(f'{self.directory}/fp1.csv', index=False)
        self.fp2.to_csv(f'{self.directory}/fp2.csv', index=False)
        self.fp3.to_csv(f'{self.directory}/fp3.csv', index=False)
        self.quali.to_csv(f'{self.directory}/quali.csv', index=False)

    def get_fp(self, session_num):
        session = fastf1.get_session(self.year, self.event_name, f'FP{session_num}')
        session.load()
        return session.laps.pick_quicklaps()

    def get_qualifying(self):
        session = fastf1.get_session(self.year, self.event_name, 'Q')
        session.load()
        quali = session.results[['Abbreviation', 'Position', 'TeamName','TeamColor', 'Q1', 'Q2', 'Q3']]
        quali['Q1'] = pd.to_timedelta(quali['Q1']).dt.total_seconds()
        quali['Q2'] = pd.to_timedelta(quali['Q2']).dt.total_seconds()
        quali['Q3'] = pd.to_timedelta(quali['Q3']).dt.total_seconds()
        # only take time from the last session the driver participated in
        quali['Time'] = quali.apply(lambda row: row['Q3'] if pd.notna(row['Q3']) else (row['Q2'] if pd.notna(row['Q2']) else row['Q1']), axis=1)
        quali = quali.drop(columns=['Q1', 'Q2', 'Q3'])
        return quali.reset_index(drop=True)

    def filter_fp(self, session, compounds=None):
        # Get the session data
        if session == 1:
            sesh = self.fp1
        elif session == 2:
            sesh = self.fp2
        elif session == 3:
            sesh = self.fp3

        # Filter by compounds if provided
        if compounds is not None:
            sesh = sesh[sesh['Compound'].isin(compounds)]

        # Remove outlaps and inlaps
        sesh = sesh[sesh['PitOutTime'].isna() & sesh['PitInTime'].isna()]

        return sesh
    
    def combine_fp(self, compounds = None):
        fp1 = self.filter_fp(1, compounds)
        fp2 = self.filter_fp(2, compounds)
        fp3 = self.filter_fp(3, compounds)

        # combine all 3 fp sessions
        combined = pd.concat([fp1, fp2, fp3])
        combined['LapTime'] = pd.to_timedelta(combined['LapTime']).dt.total_seconds()
        return combined

    def get_driver_stats(self):
        """
        Retrieve driver statistics using only FastF1 API
        """
        try:
            # Create driver-team mapping from qualifying results
            driver_team = self.quali[['Abbreviation', 'TeamName']]
            driver_team.columns = ['Driver', 'Team']

            # Get historical session data to calculate additional stats
            try:
                # Fetch previous sessions for the same event in prior years
                historical_sessions = []
                for past_year in range(self.year-3, self.year):
                    try:
                        past_session = fastf1.get_session(past_year, self.event_name, 'Q')
                        past_session.load()
                        historical_sessions.append(past_session)
                    except:
                        continue

                # Calculate historical qualifying performance
                if historical_sessions:
                    historical_positions = []
                    for session in historical_sessions:
                        historical_positions.append(session.results['Position'].to_dict())

                    # Calculate average historical position
                    drivers_avg_position = {}
                    for d in self.drivers:
                        positions = [hist.get(d, 20) for hist in historical_positions]
                        drivers_avg_position[d] = np.mean(positions) if positions else 20

                    driver_team['quali_avg'] = driver_team['Driver'].map(drivers_avg_position)
                else:
                    driver_team['quali_avg'] = 20

            except Exception as hist_error:
                print(f"Error retrieving historical data: {hist_error}")
                driver_team['quali_avg'] = 20

            # Add dummy points columns (since we can't get from Ergast)
            driver_team['Driver_points'] = 0
            driver_team['Team_points'] = 0

            return driver_team

        except Exception as e:
            print(f"Error in get_driver_stats: {e}")
            # Return a basic dataframe with zero values if everything fails
            return pd.DataFrame({
                'Driver': self.quali['Abbreviation'],
                'Driver_points': 0,
                'Team_points': 0,
                'quali_avg': 20
            })

    def preprocess(self, compounds=['SOFT']):
        """
        Preprocesses the combined free practice data to compute statistical features for each driver.
        Returns a dataset with one row per driver containing features and the target variable.
        """
        from scipy.stats import kurtosis
        
        # Combine all FP sessions, filtered by compounds if specified
        df = self.combine_fp(compounds=compounds)
        
        # Group by 'Driver' and compute statistical features
        features = df.groupby('Driver').agg({
            'LapTime': ['mean', 'std', 'min', 'max', 'median', 'skew', lambda x: kurtosis(x, bias=False) ,'count'],
            'SpeedST': ['mean', 'std']
        })
        
        # Flatten the MultiIndex columns
        features.columns = ['_'.join(col).strip() for col in features.columns.values]
        features.rename(columns={'LapTime_<lambda_0>': 'LapTime_kurtosis'}, inplace=True)
        
        # Reset index to make 'Driver' a column
        features = features.reset_index()
        
        # Prepare qualifying results with the target variable
        # Rename 'Abbreviation' to 'Driver' to match with 'features' DataFrame
        quali_results = self.quali[['Abbreviation', 'Position']].rename(columns={'Abbreviation': 'Driver'})
        
        # Merge features and qualifying results on 'Driver'
        dataset = pd.merge(features, quali_results, on='Driver')

        # merge driver stats to it as well
        dataset = pd.merge(dataset, self.get_driver_stats(), on='Driver', how='left')
        
        # Return the final dataset
        return dataset
    

# # round 1 and 15 are missing driver points these were rookie standins.
# race_name =[]
# race_data = []
# for event in schedule['EventName'].to_list()[:-1]:
#     race_name.append(event)
#     race_data.append(Race(event).preprocess())

# race_data[0]
# Ensure you have enabled FastF1 caching
fastf1.Cache.enable_cache('/Users/EmilyHuang/Desktop/NYU/fall 24/predictive/group project')

# Assuming 'schedule' is a DataFrame with 'EventName' and 'Year' columns
race_name = []
race_data = []

for event in schedule['EventName'].to_list()[:-1]:
    try:
        race_name.append(event)
        race_data.append(Race(event, year=2024).preprocess())
    except Exception as e:
        print(f"Could not process {event}: {e}")



core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '44', '77', '16', '10', '3', '4', '55', '14', '18', '11', '99', '22', '7', '63', '31', '6', '5', '47', '9']
core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.4.4]
req            INFO 	Using cache

In [25]:
def split_xy(drivers, y, X, race_ids, test_id):
    
    train_filter = race_ids != test_id
    X_train = X[train_filter]
    y_train = y[train_filter]

    test_filter = race_ids == test_id
    X_test = X[test_filter]
    y_test = y[test_filter]

    drivers_test = drivers[race_ids == test_id]

    return X_train, y_train, X_test, y_test, drivers_test

In [26]:
driver_list = []
group_size_list = []
y_list = []
X_list = []
race_ids = []
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()

race_id = 0 # will correspond to values in race_names
for race in race_data:
    # get driver ordering
    driver = race['Driver'].values
    driver_list.append(driver)
    
    # get group size
    group_size = len(driver)
    group_size_list.append(group_size)
    
    # get preference score
    y = race['Position']
    y = y.max() - y + 1 # Reverse the order so that the fastest driver has the highest value
    y_list.append(y)
    
    # standardize X
    X = race.drop(columns=['Driver', 'Position'])
    
    # Select only numeric columns
    numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
    X_numeric = X[numeric_columns]
    
    # Scale only the numeric columns
    X_scaled = scaler.fit_transform(X_numeric)
    
    # Create a DataFrame with scaled values
    X_scaled_df = pd.DataFrame(X_scaled, columns=numeric_columns, index=X.index)
    
    X_list.append(X_scaled_df)
    
    # add race id
    race_ids.append([race_id]*group_size)
    race_id += 1

print(len(driver_list), len(group_size_list), len(y_list), len(X_list), len(race_ids))

17 17 17 17 17


In [ ]:
def plot_fp(race_names, compounds=['SOFT']):
    for name in race_names:
        Race(name).plot_fp(compounds=compounds)


from sklearn.preprocessing import MinMaxScaler, StandardScaler

def split_xy(drivers, y, X, race_ids, test_id):
    
    train_filter = race_ids != test_id
    X_train = X[train_filter]
    y_train = y[train_filter]

    test_filter = race_ids == test_id
    X_test = X[test_filter]
    y_test = y[test_filter]

    drivers_test = drivers[race_ids == test_id]

    return X_train, y_train, X_test, y_test, drivers_test

def predict_rankings(model, X_test, y_test, drivers_test):
    y_pred = model.predict(X_test)


    # transform y_test back to original scale
    y_true = y_test.max() - y_test + 1
    predicted_rankings = (-y_pred).argsort().argsort() + 1


    results_df = pd.DataFrame({'PredictedRank': predicted_rankings, 'ActualPosition': y_true, 'driver': drivers_test})
    results_df = results_df.sort_values('ActualPosition').set_index('driver')
    results_df['Distance'] =np.abs(results_df['PredictedRank'] - results_df['ActualPosition']) 
    return results_df


def evaluate_model(results_df, race_name):
    # Evaluate the model
    from scipy.stats import spearmanr, kendalltau
    from sklearn.metrics import mean_absolute_error

    mean_ab_error = mean_absolute_error(results_df['ActualPosition'], results_df['PredictedRank'])
    # mse = np.sqrt(np.mean((results_df['ActualPosition'] - results_df['PredictedRank'])**2))
    spearman_corr, _ = spearmanr(results_df['PredictedRank'], results_df['ActualPosition'])
    # kendall_corr, _ = kendalltau(results_df['PredictedRank'], results_df['ActualPosition'])

    return {'Race' : race_name,
            'mean_ab_error': mean_ab_error, 
            'spearman_corr': spearman_corr
            }


def filter_for_race(race_to_predict=None, driver_list=driver_list, group_size_list=group_size_list, y_list=y_list, X_list=X_list, race_ids=race_ids):
    print(f"Filtering for race: {race_to_predict}")
    print(f"Number of races in data: {len(X_list)}")

    if race_to_predict is None or race_to_predict > len(X_list):
        raise ValueError(f"Invalid race_to_predict value: {race_to_predict}")

    try:
        drivers_combined = np.concatenate(driver_list[:race_to_predict+1])
        group_size_list = np.array(group_size_list[:race_to_predict+1])
        y_combined = np.concatenate(y_list[:race_to_predict+1])
        X_combined = np.concatenate(X_list[:race_to_predict+1])
        race_ids_combined = np.concatenate(race_ids[:race_to_predict+1])

        print(f"Shapes after concatenation:")
        print(f"drivers_combined: {drivers_combined.shape}")
        print(f"y_combined: {y_combined.shape}")
        print(f"X_combined: {X_combined.shape}")
        print(f"race_ids_combined: {race_ids_combined.shape}")

        return drivers_combined, y_combined, X_combined, race_ids_combined, group_size_list

    except ValueError as e:
        print(f"Error in concatenation: {e}")
        print(f"Lengths of lists:")
        print(f"driver_list: {len(driver_list)}")
        print(f"group_size_list: {len(group_size_list)}")
        print(f"y_list: {len(y_list)}")
        print(f"X_list: {len(X_list)}")
        print(f"race_ids: {len(race_ids)}")
        raise

# plot data for model
def display_results(summary):
    # bar chart where x is the event name, and the y values are all the scores
    summary_df = pd.DataFrame(summary).set_index('Race')
    print('Means:')
    print(summary_df.mean())
    results = schedule[['RoundNumber', 'EventName']].merge(summary_df, left_on='EventName', right_index=True).set_index('EventName')
    results.sort_values('RoundNumber', inplace=True)
    results.drop(columns='RoundNumber', inplace=True)
    fig = go.Figure()
    for col in summary_df.columns:
        fig.add_trace(go.Bar(x=summary_df.index, y=summary_df[col], name=col))

    error = summary_df[['mean_ab_error', 'spearman_corr']]
    error['cum_mae'] = summary_df['mean_ab_error'].cumsum() / range(1, len(summary_df) + 1)
    error['cum_spearman'] = summary_df['spearman_corr'].cumsum() / range(1, len(summary_df) + 1)

    fig.add_trace(go.Scatter(x=summary_df.index, y=error['cum_mae'], mode='lines+markers', name='Cumulative MAE', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=summary_df.index, y=error['cum_spearman'], mode='lines+markers', name='Cumulative Spearman', line=dict(color='red')))
    
    fig.update_layout(barmode='group', xaxis_title='Event', yaxis_title='Score')
    fig.update_layout(title='Model Evaluation Scores by Event')
    fig.show()
    return summary_df


In [22]:
driver_list = []
group_size_list = []
y_list = []
X_list = []
race_ids = []

scaler = MinMaxScaler()

race_id = 0 # will correspond to values in race_names
for race in race_data:
    # get driver ordering
    driver = race['Driver'].values
    driver_list.append(driver)
    
    # get group size
    group_size = len(driver)
    group_size_list.append(group_size)
    
    # get preference score
    y = race['Position']
    y = y.max() - y + 1 # Reverse the order so that the fastest driver has the highest value
    y_list.append(y)
    
    # standardize X
    X = race.drop(columns=['Driver', 'Position'])
    
    # Select only numeric columns
    numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
    X_numeric = X[numeric_columns]
    
    # Scale only the numeric columns
    X_scaled = scaler.fit_transform(X_numeric)
    
    # Create a DataFrame with scaled values
    X_scaled_df = pd.DataFrame(X_scaled, columns=numeric_columns, index=X.index)
    
    X_list.append(X_scaled_df)
    
    # add race id
    race_ids.append([race_id]*group_size)
    race_id += 1

def filter_for_race(race_to_predict=None, driver_list=driver_list ,group_size_list=group_size_list ,y_list=y_list ,X_list=X_list ,race_ids=race_ids):

    drivers_combined = np.concatenate(driver_list[:race_to_predict+1])
    group_size_list = np.array(group_size_list[:race_to_predict+1])
    y_combined = np.concatenate(y_list[:race_to_predict+1])
    X_combined = np.concatenate(X_list[:race_to_predict+1])
    race_ids_combined = np.concatenate(race_ids[:race_to_predict+1])


    return drivers_combined, y_combined, X_combined, race_ids_combined, group_size_list



len(driver_list), len(group_size_list), len(y_list), len(X_list), len(race_ids)


print(len(driver_list), len(group_size_list), len(y_list), len(X_list), len(race_ids))

17 17 17 17 17


In [43]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from scipy.stats import spearmanr
import plotly.graph_objects as go

class RankNetPredictor:
    def __init__(self):
        self.model = None
        self.scaler = StandardScaler()

    def build_model(self, input_dim):
        """
        Builds the RankNet model using shared layers for pairwise ranking.
        """
        l2_reg = 0.0001
        dropout_rate = 0.2
        base_network = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2_reg)),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(1)  # Single score output
        ])
        

        # Input pairs
        input1 = tf.keras.Input(shape=(input_dim,))
        input2 = tf.keras.Input(shape=(input_dim,))

        # Pass both inputs through the same network
        score1 = base_network(input1)
        score2 = base_network(input2)

        # Calculate the score difference
        score_diff = tf.keras.layers.Subtract()([score1, score2])

        # Convert to probabilities using a sigmoid
        output = tf.keras.layers.Activation('sigmoid')(score_diff)

        # Build the model
        self.model = tf.keras.Model(inputs=[input1, input2], outputs=output)
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
    

    def _generate_pairs(self, X, y):
        """
        Generates pairwise data from the training set.
        """
        X1, X2, labels = [], [], []
        for i in range(len(y)):
            for j in range(len(y)):
                if i != j:
                    X1.append(X[i])
                    X2.append(X[j])
                    labels.append(1 if y[i] < y[j] else 0)  # 1 if i ranks higher than j
        return np.array(X1), np.array(X2), np.array(labels)

    def preprocess_data(self, drivers, y, X, race_ids, race_to_predict):
        """
        Splits data into training and test sets, only using past data for training.
        """
        # Training data includes only past races
        train_filter = race_ids < race_to_predict
        X_train = X[train_filter]
        y_train = y[train_filter]

        # Testing data is for the current race
        test_filter = race_ids == race_to_predict
        X_test = X[test_filter]
        y_test = y[test_filter]
        drivers_test = drivers[test_filter]

        # Scale the features
        X_train = self.scaler.fit_transform(X_train)
        X_test = self.scaler.transform(X_test)

        return X_train, y_train, X_test, y_test, drivers_test

    def train(self, X_train, y_train, epochs=30, batch_size=64):
        """
        Trains the RankNet model on pairwise data.
        """
        # Generate pairwise training data
        X1_train, X2_train, labels = self._generate_pairs(X_train, y_train)
        history = self.model.fit(
            [X1_train, X2_train], labels,
            epochs=epochs,
            batch_size=batch_size,
            validation_split=0.2,
            verbose=1
        )
        return history

    def predict(self, X):
        """
        Predicts scores for the given feature matrix.
        """
        scores = self.model.predict([X, X])[:, 0]
        return scores

    def evaluate_model(self, results_df, race_name):
        """
        Evaluates the model's performance for a single race.
        """
        # Remove any remaining NaN values
        results_df = results_df.dropna(subset=['ActualPosition', 'PredictedRank'])
        
        if results_df.empty:
            print(f"Warning: No valid data for race {race_name}")
            return {
                'Race': race_name,
                'mean_ab_error': np.nan,
                'spearman_corr': np.nan
            }
        
        mean_ab_error = mean_absolute_error(results_df['ActualPosition'], results_df['PredictedRank'])
        spearman_corr, _ = spearmanr(results_df['PredictedRank'], results_df['ActualPosition'])

        return {
            'Race': race_name,
            'mean_ab_error': mean_ab_error,
            'spearman_corr': spearman_corr
        }
    
    # plot data for model
    def display_results(self, summary):
        # bar chart where x is the event name, and the y values are all the scores
        summary_df = pd.DataFrame(summary).set_index('Race')
        print('Means:')
        print(summary_df.mean())
        results = schedule[['RoundNumber', 'EventName']].merge(summary_df, left_on='EventName', right_index=True).set_index('EventName')
        results.sort_values('RoundNumber', inplace=True)
        results.drop(columns='RoundNumber', inplace=True)
        fig = go.Figure()
        fig.add_trace(go.Bar(x=summary_df.index, y=summary_df['mean_ab_error'], name='MARE'))
        fig.add_trace(go.Bar(x=summary_df.index, y=summary_df['spearman_corr'], name='Spearman'))

        error = summary_df[['mean_ab_error', 'spearman_corr']]
        error['cum_mae'] = summary_df['mean_ab_error'].cumsum() / range(1, len(summary_df) + 1)
        error['cum_spearman'] = summary_df['spearman_corr'].cumsum() / range(1, len(summary_df) + 1)

        fig.add_trace(go.Scatter(x=summary_df.index, y=error['cum_mae'], mode='lines+markers', name='Cumulative MARE', line=dict(color='blue')))
        fig.add_trace(go.Scatter(x=summary_df.index, y=error['cum_spearman'], mode='lines+markers', name='Cumulative Spearman', line=dict(color='red')))
        
        fig.update_layout(barmode='group', xaxis_title='Event', yaxis_title='Score')
        fig.update_layout(title='Model Evaluation Scores by Event')
        fig.show()
        return summary_df



In [39]:
ranking_results = {}
ranknet_predictor = RankNetPredictor()

for test_id in range(1, 17):
    print(f"\nProcessing race {test_id}")
    try:
        # Get data only up to a particular round
        drivers_combined, y_combined, X_combined, race_ids_combined, group_size_list = filter_for_race(race_to_predict=test_id)

        # Split data
        X_train, y_train, X_test, y_test, drivers_test = split_xy(drivers_combined, y_combined, X_combined, race_ids_combined, test_id)

        print(f"Training data shape: {X_train.shape}")
        print(f"Test data shape: {X_test.shape}")

        # Build the RankNet model if not already built
        if ranknet_predictor.model is None:
            ranknet_predictor.build_model(input_dim=X_train.shape[1])

        # Train the RankNet model
        ranknet_predictor.train(X_train, y_train, epochs=30, batch_size=16)

        # Predict rankings
        y_pred_scores = ranknet_predictor.predict(X_test)
        predicted_rankings = (-y_pred_scores).argsort().argsort() + 1  # Convert scores to rankings

        # Create a results DataFrame
        results_df = pd.DataFrame({
            'PredictedRank': predicted_rankings,
            'ActualPosition': y_test.max() - y_test + 1,  # Transform actual positions if needed
            'driver': drivers_test
        }).sort_values('ActualPosition').set_index('driver')

        results_df['Distance'] = np.abs(results_df['PredictedRank'] - results_df['ActualPosition'])

        # Store results
        ranking_results[race_name[test_id]] = results_df

    except Exception as e:
        print(f"Error processing race {test_id}: {e}")
        continue

# Evaluate model performance for each race
# summary = []
# for race, result in ranking_results.items():
#     summary.append(ranknet_predictor.evaluate_model(result, race))

# # Display results
# summary_df = ranknet_predictor.display_results(summary)
# print(summary_df)


Processing race 1
Training data shape: (20, 13)
Test data shape: (20, 13)
Epoch 1/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5073 - loss: 0.7011 - val_accuracy: 0.8553 - val_loss: 0.6312
Epoch 2/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7197 - loss: 0.6354 - val_accuracy: 0.8421 - val_loss: 0.5591
Epoch 3/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7258 - loss: 0.5798 - val_accuracy: 0.8421 - val_loss: 0.4981
Epoch 4/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7815 - loss: 0.5226 - val_accuracy: 0.8684 - val_loss: 0.4433
Epoch 5/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7938 - loss: 0.4890 - val_accuracy: 0.8421 - val_loss: 0.4051
Epoch 6/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8029 - loss: 0.4404 - val_accuracy: 0.8947 - val_loss: 0.3640
Epoch 7/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8149 - loss: 0.4027 - val_accuracy: 0.8816 - val_loss: 0.3342
Epoch 8/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2m

In [46]:
def display_results(summary):
    # bar chart where x is the event name, and the y values are all the scores
        summary_df = pd.DataFrame(summary).set_index('Race')
        print('Means:')
        print(summary_df.mean())
        results = schedule[['RoundNumber', 'EventName']].merge(summary_df, left_on='EventName', right_index=True).set_index('EventName')
        results.sort_values('RoundNumber', inplace=True)
        results.drop(columns='RoundNumber', inplace=True)
        fig = go.Figure()
        fig.add_trace(go.Bar(x=summary_df.index, y=summary_df['mean_ab_error'], name='MARE'))
        fig.add_trace(go.Bar(x=summary_df.index, y=summary_df['spearman_corr'], name='Spearman'))

        error = summary_df[['mean_ab_error', 'spearman_corr']]
        error['cum_mae'] = summary_df['mean_ab_error'].cumsum() / range(1, len(summary_df) + 1)
        error['cum_spearman'] = summary_df['spearman_corr'].cumsum() / range(1, len(summary_df) + 1)

        fig.add_trace(go.Scatter(x=summary_df.index, y=error['cum_mae'], mode='lines+markers', name='Cumulative MARE', line=dict(color='blue')))
        fig.add_trace(go.Scatter(x=summary_df.index, y=error['cum_spearman'], mode='lines+markers', name='Cumulative Spearman', line=dict(color='red')))
        
        fig.update_layout(barmode='group', xaxis_title='Event', yaxis_title='Score')
        fig.update_layout(title='Model Evaluation Scores by Event')
        fig.show()
        return summary_df
# Evaluate model performance for each race
summary = []
for race, result in ranking_results.items():
    summary.append(ranknet_predictor.evaluate_model(result, race))

# Display results
summary_df = display_results(summary)
print(summary_df)

Means:
mean_ab_error    6.357544
spearman_corr    0.068555
dtype: float64


                           mean_ab_error  spearman_corr
Race                                                   
Saudi Arabian Grand Prix        6.600000       0.012030
Australian Grand Prix           5.263158       0.303509
Japanese Grand Prix             6.800000      -0.078195
Emilia Romagna Grand Prix       5.700000       0.112782
Monaco Grand Prix               5.900000       0.224060
Canadian Grand Prix             5.900000       0.221053
Spanish Grand Prix              7.800000      -0.392481
British Grand Prix              6.900000       0.091729
Hungarian Grand Prix            7.100000       0.012030
Belgian Grand Prix              7.000000      -0.163910
Dutch Grand Prix                     NaN            NaN
Italian Grand Prix              5.500000       0.261654
Azerbaijan Grand Prix           6.300000       0.154887
Singapore Grand Prix            6.500000       0.066165
Mexico City Grand Prix          6.100000       0.118797
Las Vegas Grand Prix            6.000000       0

In [ ]:
summary = []
for race, result in ranking_results.items():
    summary.append(evaluate_model(result, race))

summary_df = display_results(summary)
summary_df


In [ ]:
ranknet_predictor.model.summary()

Model: "functional_20"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_18      │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_19      │ (None, 13)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_6        │ (None, 1)         │      3,009 │ input_layer_18[0… │
│ (Sequential)        │                   │            │ input_layer_19[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ subtract_6          │ (None, 1)         │          0 │ sequential_6[0][… │
│ (Subtract)          │                   │            │ sequential_6[1][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 1)         │          0 │ subtract_6[0][0]  │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,029 (35.27 KB)

 Trainable params: 3,009 (11.75 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,020 (23.52 KB)

In [48]:
import plotly.graph_objects as go
sum_copy = summary_df. copy ()
sum_copy.rename(columns={'mean_ab_error': 'MARE', 'spearman_corr': 'Spearman'}, inplace=True)
sum_copy.reset_index(inplace=True)
sum_copy[' Cumulative MARE'] = sum_copy['MARE']. cumsum() / range(1, len(sum_copy) + 1)
sum_copy['Cumulative Spearman'] = sum_copy['Spearman'].cumsum() / range(1, len(sum_copy) + 1)
sum_copy= sum_copy.round(2)
fig = go.Figure (data= [go.Table(
    header=dict (values=list (sum_copy.columns) ,
    align='left'),
    cells=dict(values=[sum_copy[col] for col in sum_copy.columns], align='left'))
    ], layout=go.Layout(width=1090, height=850))
fig.show()